<a href="https://colab.research.google.com/github/tsarnadelis/OperationalResearchProject/blob/main/OR2024_%CE%A4%CF%83%CE%B1%CF%81%CE%BD%CE%B1%CE%B4%CE%AD%CE%BB%CE%B7%CF%82_10388.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Operational Research 2024 ECE AUTH
## Τσαρναδέλης Αθανάσιος Γρηγόριος 10388
### atsarnad@ece.auth.gr

## Προγραμματισμός Προσγειώσεων Πτήσεων

### Packages
Χρησιμοποιώ την βιβλιοθήκη GurobiPy για την βελτιστοποίηση ,και την tabulate για να παρουσιάσω τα αποτελέσματα.

In [ ]:
%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
from tabulate import tabulate

### Δεδομένα
Ορίζω τα δεδομένα που δίνονται στην εκφώνηση, η επεξήγηση της κάθε μεταβλητής φαίνεται παρακάτω.


In [ ]:
# Δεδομένα
M = 10^10 # Big M, για εκεί που δεν υπάρχουν δεδομένα
n = 10  # Αριθμός αεροσκαφών
e = [129, 195, 89, 96, 110, 120, 124, 126, 135, 160]  # Νωρίτερος χρόνος άφιξης
est = [155, 258, 96, 106, 123, 135, 138, 140, 150, 180] # Εκτιμώμωνος χρόνος άφιξης
l = [559, 744, 510, 521, 555, 576, 577, 573, 591, 657]  # Αργότερος χρόνος άφιξης
c_early = [10, 10, 30, 30, 30, 30, 30, 30, 30, 30]  # Κόστος πρόωρης άφιξης ανά λεπτό
c_late = [10, 10, 30, 30, 30, 30, 30, 30, 30, 30] # Κόστος καθυστερημένης άφιξης ανά λεπτό
g = [[M, 3, 15, 15, 15, 15, 15, 15, 15, 15], # Ελάχιστος χρόνος μεταξύ προσγειώσεων
     [3, M, 15, 15, 15, 15, 15, 15, 15, 15],
     [15, 15, M, 8, 8, 8, 8, 8, 8, 8],
     [15, 15, 8, M, 8, 8, 8, 8, 8, 8],
     [15, 15, 8, 8, M, 8, 8, 8, 8, 8],
     [15, 15, 8, 8, 8, M, 8, 8, 8, 8],
     [15, 15, 8, 8, 8, 8, M, 8, 8, 8],
     [15, 15, 8, 8, 8, 8, 8, M, 8, 8],
     [15, 15, 8, 8, 8, 8, 8, 8, M, 8],
     [15, 15, 8, 8, 8, 8, 8, 8, 8, M]]

### Ορισμός μοντέλου και μεταβλητές
Ορίζω το μοντέλο μου και τις μεταβλητές που χρειάζομαι. Η μεταβλητή t είναι μεταβλητή απόφασης, ενώ οι άλλες δύο αφορούν τα πρόστιμα και θα με βοηθήσουν στον ορισμό των περιορισμών.

In [ ]:
# Δημιουργία μοντέλου
model = gp.Model("landing_schedule")

# Μεταβλητές
t = model.addVars(n, vtype=GRB.CONTINUOUS, name="t")
early_penalty = model.addVars(n, vtype=GRB.CONTINUOUS, name="early_penalty")
late_penalty = model.addVars(n, vtype=GRB.CONTINUOUS, name="late_penalty")

### Ελαχιστοποίηση
Ορίζω την συνάρτηση που θα ελαχιστοποιήσω, που είναι το συνολικό ποσό των προστίμων, δηλαδή το πρόστιμο πρόωρης άφιξης επί τα λεπτά πρόωρης άφιξης, συν το πρόστιμο αργοπορημένης άφιξης επί τα λεπτά που αργοπόρησε το αεροσκάφος.

In [ ]:
# Στόχος: Ελαχιστοποίηση του συνολικού προστίμου
model.setObjective(gp.quicksum(c_early[i] * early_penalty[i] + c_late[i] * late_penalty[i] for i in range(n)), GRB.MINIMIZE)

### Περιορισμοί και βελτιστοποίηση
1. Ο χρόνος άφιξης του αεροσκάφους πρέπει να είναι μεγαλύτερος από τον ελάχιστο και μικρότερος από τον μέγιστο
2. Τα early_penalty και late_penalty είναι η διαφορές των χρόνων από τον εκτιμώμενο και πρέπει να είναι θετικές
3. Η διαφορά άφιξης ενός αεροσκάφους με οποιδήποτε άλλο πρέπει να είναι μεγαλύτερη ή ίση από αυτήν που δίνεται στην εκφώνηση

Με τους παραπάνω περιορισμούς κάνω βελτιστοποίηση.

In [ ]:
# Περιορισμοί:
# 1. Νωρίτεροι και αργότεροι χρόνοι άφιξης
for i in range(n):
    model.addConstr(t[i] >= e[i], name=f"earliest_time_{i}")
    model.addConstr(t[i] <= l[i], name=f"latest_time_{i}")

# 2. Υπολογισμός προστίμου πρόωρης και καθυστερημένης άφιξης
for i in range(n):
    model.addConstr(early_penalty[i] >= est[i] - t[i], name=f"early_penalty_{i}")
    model.addConstr(early_penalty[i] >= 0, name=f"non_negative_early_penalty_{i}")
    model.addConstr(late_penalty[i] >= t[i] - est[i], name=f"late_penalty_{i}")
    model.addConstr(late_penalty[i] >= 0, name=f"non_negative_late_penalty_{i}")

# 3. Ελάχιστος χρόνος μεταξύ των προσγειώσεων
for i in range(n):
    for j in range(i + 1, n):
        model.addConstr(t[j] - t[i] >= g[i][j], name=f"min_gap_{i}_{j}")

# Βελτιστοποίηση του μοντέλου
# model.setParam('OutputFlag',False) # Hush gurobi solver output
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 105 rows, 30 columns and 170 nonzeros
Model fingerprint: 0x1381f806
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 7e+02]
Presolve removed 48 rows and 8 columns
Presolve time: 0.01s
Presolved: 57 rows, 22 columns, 114 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.154194e+02   0.000000e+00      0s
      19    2.5710000e+04   0.000000e+00   0.000000e+00      0s

Solved in 19 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.571000000e+04


### Αποτελέσματα
Τέλος, εμφανίζω τα αποτελέσματα παρακάτω:

In [ ]:

# Εμφάνιση των αποτελεσμάτων
if model.status == GRB.OPTIMAL:
    landing_times = model.getAttr('x', t)
    results = []
    for i in range(n):
        results.append([f"Αεροσκάφος {i + 1}", f"{landing_times[i]:3.0f} λεπτά"])
    print(tabulate(results , headers=["Αεροσκάφος", "Προγραμματισμένος Χρόνος Προσγείωσης"], tablefmt="grid"))
else:
    print("Δεν βρέθηκε βέλτιστη λύση!")

+---------------+----------------------------------------+
| Αεροσκάφος    | Προγραμματισμένος Χρόνος Προσγείωσης   |
+===============+========================================+
| Αεροσκάφος 1  | 155 λεπτά                              |
+---------------+----------------------------------------+
| Αεροσκάφος 2  | 195 λεπτά                              |
+---------------+----------------------------------------+
| Αεροσκάφος 3  | 210 λεπτά                              |
+---------------+----------------------------------------+
| Αεροσκάφος 4  | 218 λεπτά                              |
+---------------+----------------------------------------+
| Αεροσκάφος 5  | 226 λεπτά                              |
+---------------+----------------------------------------+
| Αεροσκάφος 6  | 234 λεπτά                              |
+---------------+----------------------------------------+
| Αεροσκάφος 7  | 242 λεπτά                              |
+---------------+---------------------------------------

## Σχεδιασμός παράδοσης πετρελαίου




### Δεδομένα
Ορίζω τα δεδομένα που δίνονται στην εκφώνηση, η επεξήγηση της κάθε μεταβλητής φαίνεται παρακάτω.

In [ ]:
# Δεδομένα
locations = ['Ω', 'Α', 'Β', 'Γ', 'Δ', 'Ε', 'ΣΤ'] # Οι τοποθεσίες του προβλήματος
demand = [0, 14000, 3000, 6000, 16000, 15000, 5000] # Ζήτηση σε λίτρα
distances = [   # Αποστάσεις μεταξύ πόλεων
    [0, 148, 55, 32, 70, 140, 73],
    [148, 0, 93, 180, 99, 12, 72],
    [55, 93, 0, 85, 20, 83, 28],
    [32, 180, 85, 0, 100, 174, 99],
    [70, 99, 20, 100, 0, 85, 49],
    [140, 12, 83, 174, 85, 0, 73],
    [73, 72, 28, 99, 49, 73, 0]
]
capacity = 39000 # Χωρητικότητα δεξαμενόπλοιου

### Ορισμός μοντέλου και μεταβλητές
Ορίζω το μοντέλο μου και τις μεταβλητές που χρειάζομαι. Η μεταβλητή t είναι μεταβλητή απόφασης, ενώ οι άλλες δύο αφορούν τα πρόστιμα και θα με βοηθήσουν στον ορισμό των περιορισμών.

In [ ]:
# Δημιουργία του μοντέλου
model = gp.Model("fuel_delivery")

# Μεταβλητές
num_locations = len(locations) # Αριθμός τοποθεσιών
x = model.addVars(num_locations, num_locations, vtype=GRB.BINARY, name='x')
u = model.addVars(num_locations, vtype=GRB.INTEGER, name='u')

### Ελαχιστοποίηση
Ορίζω την συνάρτηση που θα ελαχιστοποιήσω, που είναι ο ελάχιστος αριθμός χιλιομέτρων για να ικανοποιηθεί η ζήτηση, δηλαδή οι διαδρομές που κάνω επί τις αποστάσεις των πόλεων της εκάστοτε διαδρομής.

In [ ]:
# Στόχος
model.setObjective(gp.quicksum(distances[i][j] * x[i, j] for i in range(num_locations) for j in range(num_locations)), GRB.MINIMIZE)

### Περιορισμοί και βελτιστοποίηση
1. Ακριβώς ένα όχημα φτάνει σε κάθε πόλη και ακριβώς ένα αναχωρεί
2. Η ποσότητα καυσίμου στην αρχή της διαδρομής πρέπει να είναι μικρότερη της χωρητικότητας
3. Κάθε κλειστή διαδρομή πρέπει να περιλαμβάνει το διυλιστήριο
4. Το φορτίο μετά την εξυπηρέτηση του πελάτη i είναι τουλάχιστον η ζήτηση του πελάτη i και δεν υπερβαίνει ποτέ την μέγιστη χωρητικότητα

Με τους παραπάνω περιορισμούς κάνω βελτιστοποίηση.

In [ ]:
# Περιορισμοί:
# 1. Ακριβώς ένα όχημα φτάνει σε κάθε πόλη και ακριβώς ένα αναχωρεί
for j in range(1, num_locations):
    model.addConstr(gp.quicksum(x[i, j] for i in range(num_locations) if i != j) == 1)
    model.addConstr(gp.quicksum(x[j, k] for k in range(num_locations) if k != j) == 1)

# 2. Η ποσότητα καυσίμου στην αρχή της διαδρομής πρέπει να είναι μικρότερη της χωρητικότητας
model.addConstr(gp.quicksum(demand[j] * x[0, j] for j in range(1, num_locations)) <= capacity)

# 3. Κάθε κλειστή διαδρομή πρέπει να περιλαμβάνει το διυλιστήριο
for i in range(1, num_locations):
    for j in range(1, num_locations):
        if i != j:
            model.addConstr(u[i] - u[j] + capacity * x[i, j] <= capacity - demand[j])

# 4. Το φορτίο μετά την εξυπηρέτηση του πελάτη i είναι τουλάχιστον η ζήτηση του πελάτη i και δεν υπερβαίνει ποτέ την μέγιστη χωρητικότητα
for i in range(1, num_locations):
    model.addConstr(u[i] >= demand[i])
    model.addConstr(u[i] <= capacity)

# Βελτιστοποίηση
model.setParam('OutputFlag',False) # Hush gurobi solver output
model.optimize()


### Αποτελέσματα
Τέλος, εμφανίζω τα αποτελέσματα παρακάτω:

In [ ]:
# Αποτελέσματα
if model.status == GRB.OPTIMAL:
    print('Ελάχιστο κόστος για εξυπηρέτηση της ζήτησης: %g km' % model.objVal)
    solution = model.getAttr('x', x)
    print('Βέλτιστες διαδρομές')
    for i in range(num_locations):
        for j in range(num_locations):
            if solution[i, j] > 0.5:
                print('%s -> %s' % (locations[i], locations[j]))
else:
    print('Δεν βρέθηκε βέλτιστη λύση!')